In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Список маршрутов автобусов

In [ ]:
url = "https://kudikina.ru/spb/bus/"

response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, "html.parser")

bus_list = []
bus_links = []

bus_items = soup.find_all("a", class_="bus-item bus-icon")
for item in bus_items:
    bus_number = item.text.strip()
    # bus_route = item.find("span").text.strip()
    href_link = item["href"]
    bus_links.append(href_link)
    # bus_list.append([bus_number, bus_route, href_link])
    bus_list.append([bus_number, href_link])

In [ ]:
bus_list

In [ ]:
bus_links

# Ссылки на автобусы

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/bus_list full.csv")


bus_links = data["Href Link"]


for bus_link in bus_links:
    full_link = f"https://kudikina.ru{bus_link}"
    print(full_link)


In [ ]:
data["new_lınk"] = "https://kudikina.ru" + data["Href Link"]

# Остановки для маршрутов автобусов

In [ ]:
# data = pd.read_csv("/content/drive/MyDrive/bus_list full.csv")


# bus_links = data["Href Link"]


for bus_link in bus_links:
    full_link = f"https://kudikina.ru{bus_link}/B"
    response = requests.get(full_link)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        bus_stops = soup.find_all("div", class_="bus-stop")

        print(f" stops {full_link} link:")
        for stop in bus_stops:
            stop_name = stop.find("a").text.strip()
            print(f"{stop_name}")

## Скачать данные

При работе с Google Colab

In [ ]:
import pandas as pd
from google.colab import files

df = pd.DataFrame(data)
df.to_csv("bus_stopsb.csv", index=False)

files.download('bus_stopsb.csv')

# Расписание для маршрутов и остановок

In [ ]:
# data = pd.read_csv("/content/drive/MyDrive/bus_list full.csv")
# bus_links = data["Href Link"]

def print_bus_stops(soup):
  bus_stops = soup.find("div", class_="bus-stops")

  for bus_stop in bus_stops.find_all("div", class_="row"):
    stop_name = bus_stop.find("div", class_="bus-stop").text.strip()

    print(stop_name)

    table_data = bus_stop.find("table")

    if table_data is not None:
      interval_data = bus_stop.find("div" , class_="interval-times")
      print(interval_data.text.strip())
      print()
      print(table_data.text.strip())
      print()
    else:
      list_data = bus_stop.find("div" , class_="stop-times")
      if list_data is not None:
        print(list_data.text.strip())
        print()


for bus_link in bus_links:
    full_link = f"https://kudikina.ru{bus_link}/A"
    response = requests.get(full_link)

    if response.status_code == 200:
      soup = BeautifulSoup(response.text, "lxml")
      bus_menu = soup.find("ul", class_="bus-menu")
      bus_menu_count = len(list(bus_menu.find_all("li")))

      print()
      print(f"Stops for {full_link} link:")

      try:
        print_bus_stops(soup)
      except Exception as e:
        print(f"Error in extracting stops for bus {full_link}: {e}")

      if bus_menu_count > 1:
        backwards_full_link = f"https://kudikina.ru{bus_link}/B"
        response = requests.get(backwards_full_link)

        if response.status_code == 200:
          soup = BeautifulSoup(response.text, "lxml")

          print()
          print(f"Stops for {backwards_full_link} link:")

          try:
            print_bus_stops(soup)
          except Exception as e:
            print(f"Error in extracting stops for bus {backwards_full_link}: {e}")

    else:
      print(f"Failed to retrieve data for {full_link}")
